In [1]:
import numpy as np

def points_from_file(path):
    f = open(path,'r')
    lines = f.readlines()
    points = np.zeros((len(lines),len(lines[0].split()))) # Numpy array to store points
    i = 0
    for line in lines:
        points[i] = list(map(float, line.split())) # Points read from file and stored in points
        i += 1
    return points

def initial_descriptor(points, dim):
    descriptor = np.zeros((points.shape[0],dim))
    for i in range(points.shape[0]):
        point_sub = points - points[i] # Subtract given point from all other points
        point_sub = np.delete(point_sub,i,0) # Remove the row with all zeros
        descriptor[i] = np.sort((point_sub[:,0]**2 + point_sub[:,1]**2 + point_sub[:,2]**2))[:dim] # Get the descriptor based on distance
    return descriptor

def matching_func(points_1, points_2, descriptor_1, descriptor_2):
    
    matched_descriptors = np.zeros(points_1.shape[0]) # if ith descriptor_1 matches with jth descriptor_2, matched_descriptors[i] = j
    for i in range(points_1.shape[0]):
        diff_descriptor = descriptor_2 - descriptor_1[i] # Subtract given descriptor from all other descriptors
        dist_descriptor = diff_descriptor**2 # Square each element
        dist_descriptor = np.sum(dist_descriptor, axis = 1) # Final Distance between two descriptors
        matched_descriptors[i] = dist_descriptor.argmin() # Matched descriptors will have minimum distance
        
    return matched_descriptors

def mmse(points_1, points_2, Rot_mat, trans): # Mean squared error
    error = 0
    for i in range(points_1.shape[0]):
        error += np.linalg.norm(np.dot(Rot_mat,points_1[i].reshape(3,1)) + trans - points_2[i].reshape(3,1))**2
    return error

def rigid_trans(points_1, points_2, matches, max_iter = 5):
    
    count = 0 # No. of iterations
    while True:    
        corresponding_points = points_2[matches.astype(int)] # Rearrange Q point cloud based on matches
        centroid_1 = np.mean(points_1, axis = 0) # Centroid of fisrt point cloud
        centroid_2 = np.mean(corresponding_points, axis = 0) # Centroid of second point cloud

        # Point cloud centroids
        x = points_1 - centroid_1  
        y = points_2 - centroid_2
        
        # Covariance Matrix with its SVD
        S = np.dot(x.T,y)
        U, E, V_T = np.linalg.svd(S)
        
        # Rotation and Translation
        R = np.dot(V_T.T,U.T)
        t = centroid_2.reshape(3,1)- np.dot(R,centroid_1.reshape(3,1))
        
        # Error
        error = mmse(points_1, points_2, R, t)
        
        # Transformed points matched by minimzing distance from Q point cloud
        new_points_2 = (np.dot(R,p_points.T) + t).T
        matches = matching_func(new_points_2, points_2, new_points_2, points_2)
        count += 1
        
        if error < 1e-4 or count == max_iter:
            break
    return R, t, new_points_2

In [2]:
# Point clouds
p_points = points_from_file('P_1.txt') 
q_points = points_from_file('Q_1.txt') 

# Descriptors
p_descriptor = initial_descriptor(p_points, 128)
q_descriptor = initial_descriptor(q_points, 128)

# Initial matches through descriptors
initial_matches = matching_func(p_points, q_points, p_descriptor, q_descriptor)

In [ ]:
# Find R, t and transformed points
R, t , transformed_points = rigid_trans(p_points,q_points, initial_matches)

#Plot all point clouds
%matplotlib widget # Installion - https://github.com/matplotlib/jupyter-matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax0 = fig.add_subplot(131, projection='3d')
ax0.scatter(p_points[:,0], p_points[:,1], p_points[:,2], c='g')
ax1 = fig.add_subplot(132, projection='3d')
ax1.scatter(q_points[:,0], q_points[:,1], q_points[:,2], c= 'b')
ax2 = fig.add_subplot(133, projection='3d')
ax2.scatter(transformed_points[:,0], transformed_points[:,1], transformed_points[:,2], c ='r')

In [4]:
print('Rotation Matrix: ')
print(R)
print()
print('Translation Vector: ')
print(t)

Rotation Matrix: 
[[ 7.66044443e-01 -2.87019658e-10 -6.42787610e-01]
 [-5.82563416e-01  4.22618262e-01 -6.94272044e-01]
 [ 2.71653783e-01  9.06307787e-01  3.23744371e-01]]

Translation Vector: 
[[1.        ]
 [1.00000001]
 [1.00000002]]


In [5]:
print('Transformed Points: \n',transformed_points,
      '\n\n','Points in Q: \n', q_points)

Transformed Points: 
 [[ -87.02132408  -88.92669117   56.38706898]
 [ -88.13931699  -90.43370947   56.30827946]
 [ -90.72974625  -94.19743731   55.54262028]
 ...
 [-139.18426503  -41.39007533   20.49891548]
 [-140.15059147  -41.61962581   19.9048074 ]
 [-138.05407819  -43.11358465   20.98970627]] 

 Points in Q: 
 [[ -87.021324  -88.926691   56.387069]
 [ -88.139317  -90.433709   56.308279]
 [ -90.729746  -94.197437   55.54262 ]
 ...
 [-139.184265  -41.390075   20.498915]
 [-140.150591  -41.619626   19.904807]
 [-138.054078  -43.113585   20.989706]]
